In [1]:
from pathlib import Path

raw_datasets = Path("datasets/battlefield")
output_dir = Path("datasets")

def create_label_file():
    label = f"""path: .
train: train_all_in_one.txt
val: val_all_in_one.txt
# test: test_all_in_one.txt

names:
  0: operator
  1: skill_ready
"""
    with open(output_dir / "label.yaml", "w") as f:
        f.write(label)

create_label_file()

In [2]:

def move_draw():
    draw_list = raw_datasets.glob("*draw.png")
    draw_dir = raw_datasets / "draw"
    draw_dir.mkdir(parents=True, exist_ok=True)
    for draw in draw_list:
        draw.rename(draw_dir / draw.name)

move_draw()

In [3]:
from PIL import Image
import random
import pypinyin

zoom = 640.0 / 720.0

def to_pinyin(text):
    return ''.join([item[0] for item in pypinyin.pinyin(
            text, style=pypinyin.NORMAL, errors='default')]).lower()

def pre_proc(save_img: bool = True):
    proc_images = raw_datasets.parent / "images"
    proc_images.mkdir(parents=True, exist_ok=True)
    proc_label = raw_datasets.parent / "labels"
    proc_label.mkdir(parents=True, exist_ok=True)
    right_move = 1138 - 640

    data_list = list(raw_datasets.glob("*.txt"))
    for idx, data in enumerate(data_list):
        with open(data, "r") as f:
            lines = f.readlines()
        if len(lines) < 2:
            continue

        stage_name = to_pinyin(lines[0].strip())
        image_path = data.with_suffix(".png")
        
        if save_img:
            image = Image.open(image_path)
            image = image.convert("RGB")
            image = image.resize((1138, 640))
            left_box = (0, 0, 640, 640)
            right_box = (right_move, 0, 1138, 640)
            left_image = image.crop(left_box)
            right_image = image.crop(right_box)

        left_label = ""
        right_label = ""
        for line_idx in range(1, len(lines)):
            oper_line = lines[line_idx]
            name, x, y, skill = oper_line.strip().split(" ")
            skill_y = (float(y) - 108) * zoom
            x = float(x) * zoom
            y = float(y) * zoom
            left = x - random.randint(38, 42)
            right = x + random.randint(38, 42)
            x = (left + right) / 2.0
            top = y + random.randint(25, 30)
            bottom = y + random.randint(45, 50)
            y = (top + bottom) / 2.0

            skill: bool = bool(int(skill))

            if x < 640:
                x = x / 640.0
                y = y / 640.0
                w = (right - left) / 640.0
                h = (bottom - top) / 640.0
                left_label += f"0 {x} {y} {w} {h}\n"
                if skill:
                    skill_y = skill_y / 640.0
                    left_label += f"1 {x} {skill_y} 0.0875 0.0875\n"
            else:
                x = (x - right_move) / 640.0
                y = y / 640.0
                w = (right - left) / 640.0
                h = (bottom - top) / 640.0
                right_label += f"0 {x} {y} {w} {h}\n"
                if skill:
                    skill_y = skill_y / 640.0
                    right_label += f"1 {x} {skill_y} 0.0875 0.0875\n"

        if left_label:
            if save_img:
                left_image.save(proc_images / f"{stage_name}_{image_path.name}_left.png")
            with open(proc_label / f"{stage_name}_{image_path.name}_left.txt", "w") as f:
                f.write(left_label)
        if right_label:
            if save_img:
                right_image.save(proc_images / f"{stage_name}_{image_path.name}_right.png")
            with open(proc_label / f"{stage_name}_{image_path.name}_right.txt", "w") as f:
                f.write(right_label)

        if idx % 1000 == 0:
            print(f"{idx} / {len(data_list)}")

pre_proc()

1000 / 12852
2000 / 12852
3000 / 12852
4000 / 12852
5000 / 12852
6000 / 12852
7000 / 12852
8000 / 12852
9000 / 12852
10000 / 12852
11000 / 12852
12000 / 12852


In [7]:
def split_train_and_val():
    proc_images = raw_datasets.parent / "images"
    image_list = list(proc_images.glob("*.png"))
    
    train_sets = []
    val_sets = []
    for idx, image in enumerate(image_list):
        if idx % 100 < 20:
            val_sets.append(image)
        else:
            train_sets.append(image)
    
    train_label = ""
    val_label = ""
    for image in train_sets:
        train_label += f"{image.absolute()}\n"
    for image in val_sets:
        val_label += f"{image.absolute()}\n"
    
    with open(output_dir / "train_all_in_one.txt", "w") as f:
        f.write(train_label)
    with open(output_dir / "val_all_in_one.txt", "w") as f:
        f.write(val_label)

split_train_and_val()
    